# Creating a Grid Integrated Plant Design Problem using IDAES Surrogates

## Import necessary packages
To begin, we need to import necessary packages. The primary packages required are:
* Pyomo : Modeling the optimization problem
* IDAES: Surrogate modeling capabilities
* Tensorflow: load trained Keras surrogates that represent market simulations

In [1]:
import json
import pandas as pd
import tensorflow

#import pyomo and its objects
from pyomo.environ import (
    ConcreteModel,
    Var,
    Constraint,
    Expression,
    NonNegativeReals,
    Block,
    value,
    Objective,
)
import pyomo.environ as pyo

# Import IDAES components
from idaes.core.util import to_json, from_json
from idaes.core.solvers import get_solver

#Import IDAES SurrogateBlock
from idaes.core.surrogate.keras_surrogate import KerasSurrogate
from idaes.core.surrogate.sampling.scaling import OffsetScaler
from idaes.core.surrogate.surrogate_block import SurrogateBlock

# Import simple rankine cycle flowsheet
import dispatches.case_studies.simple_rankine_cycle.simple_rankine_cycle as src

## Load Keras Surrogates
The first step towards assembling our optimization model is to load the pre-trained Keras neural networks (available in this DISPATCHES repository) as well as associated meta-data saved in corresponding .json files. This meta data contains input and output labels, scaling information, and the input bounds used during training.

The keras models themselves represent the results of market simulations using the Prescient Production Cost Modeling open-source package. We previously trained neural network models to predict the following outputs based on market inputs:
* Annual Revenue \[MM\$\]: (the total amount of revenue our generator earned per year)
* Annual Number of Generator Startups
* Annual Zone Output \[hours\]: (the amount of time our generator spent at different power output intervals). For this neural network, we trained 11 different zone outputs (off,0-10% of pmax,10-20% of pmax,...,90-100% of pmax)

For market inputs, our trianing data consisted of 8 primary attributes:
* p_max (the nameplate plant capacity)
* p_min_multipler (the minimum operating output as a fration of p_max)
* ramp_rate_multiplier
* minimum_up_time
* minimum_dn_multiplier
* marginal_cost
* fixed_run_cost
* startup_profile

In [2]:
# load training meta-data for each surrogate. this includes things like scaling 
# information, labels, and input bounds

with open("keras_models/models/training_parameters_revenue.json", 'rb') as f:
    rev_data = json.load(f)

with open("keras_models/models/training_parameters_zones.json", 'rb') as f:
    zone_data = json.load(f)

with open("keras_models/models/training_parameters_nstartups.json", 'rb') as f:
    nstartups_data = json.load(f)

# unpack scaling information for revenue
xm_rev = rev_data['xm_inputs']
xstd_rev = rev_data['xstd_inputs']
zm_rev = rev_data['zm_revenue']
zstd_rev = rev_data['zstd_revenue']

# unpack scaling information for number of startups
xm_nstart = nstartups_data['xm_inputs']
xstd_nstart = nstartups_data['xstd_inputs']
zm_nstartups = nstartups_data['zm_nstartups']
zstd_nstartups = nstartups_data['zstd_nstartups']

# unpack scaling information for duration at different operating outputs
xm_zones = zone_data['xm_inputs']
xstd_zones = zone_data['xstd_inputs']
zm_zones = zone_data['zm_zones']
zstd_zones = zone_data['zstd_zones']

# load surrogates from keras directories
keras_revenue = tensorflow.keras.models.load_model('keras_models/models/keras_revenue', compile=False)
keras_nstartups = tensorflow.keras.models.load_model('keras_models/models/keras_nstartups', compile=False)
keras_zones = tensorflow.keras.models.load_model('keras_models/models/keras_zones', compile=False)

## Create IDAES Keras Surrogate Objects

Now that we have our keras models loaded we can create `KerasSurrogate` objects using IDAES. These objects wrap our trained Keras Sequential models and store the additional training information we loaded such as scaling. The `KerasSurrogate` object provides a consistent interface for handling surrogate models in IDAES.

Once we have a `KerasSurrogate`, we can use it to build a mathematical model on a `SurrogateBlock`. This is shown later in this notebook.

In [3]:
# revenue Keras surrogate
# load extra meta-data for the revenue surrogate
# input_labels = list('x'+str(i) for i in range(len(xm_rev)))
# output_labels = ["z_rev"]
input_labels = rev_data['input_labels']
output_labels = rev_data['output_labels']
xmin = rev_data['xmin']
xmax = rev_data['xmax']

# create scaling objects for the neural network input and outputs
inputs_scaler = OffsetScaler(
    expected_columns=input_labels,
    offset_series = pd.Series(dict(zip(input_labels, xm_rev))),
    factor_series = pd.Series(dict(zip(input_labels, xstd_rev))),
)

outputs_scaler = OffsetScaler(
    expected_columns=output_labels,
    offset_series = pd.Series(dict(zip(output_labels, [zm_rev]))),
    factor_series = pd.Series(dict(zip(output_labels, [zstd_rev]))),
)

# store input bounds in a dictionary where keys correspond to input labels and values are 2-length tuples
# of the lower and upper bound respectively
input_bounds={input_labels[i]: (xmin[i], xmax[i]) for i in range(len(input_labels))}

# create the KerasSurrogate object using the keras neural network and associated training data
keras_revenue_surrogate = KerasSurrogate(
    keras_model=keras_revenue,
    input_labels=input_labels,
    output_labels=output_labels,
    input_bounds=input_bounds,
    input_scaler=inputs_scaler,
    output_scaler=outputs_scaler,
)

# nstartups Keras surrogate
input_labels = nstartups_data['input_labels']
output_labels = nstartups_data['output_labels']
xmin = nstartups_data['xmin']
xmax = nstartups_data['xmax']

inputs_scaler = OffsetScaler(
    expected_columns=input_labels,
    offset_series = pd.Series(dict(zip(input_labels, xm_nstart))),
    factor_series = pd.Series(dict(zip(input_labels, xstd_nstart))),
)

outputs_scaler = OffsetScaler(
    expected_columns=output_labels,
    offset_series = pd.Series(dict(zip(output_labels,[zm_nstartups]))),
    factor_series = pd.Series(dict(zip(output_labels,[zstd_nstartups]))),
)

input_bounds={input_labels[i]: (xmin[i], xmax[i]) for i in range(len(input_labels))}

keras_nstartups_surrogate = KerasSurrogate(
    keras_model=keras_nstartups,
    input_labels=input_labels,
    output_labels=output_labels,
    input_bounds=input_bounds,
    input_scaler=inputs_scaler,
    output_scaler=outputs_scaler,
)

# zone hour power output Keras surrogate
input_labels = zone_data['input_labels']
output_labels = zone_data['output_labels']
xmin = zone_data['xmin']
xmax = zone_data['xmax']

inputs_scaler = OffsetScaler(
    expected_columns=input_labels,
    offset_series = pd.Series(dict(zip(input_labels, xm_zones))),
    factor_series = pd.Series(dict(zip(input_labels, xstd_zones))),
 )

outputs_scaler = OffsetScaler(
    expected_columns=output_labels,
    offset_series = pd.Series(dict(zip(output_labels, zm_zones))),
    factor_series = pd.Series(dict(zip(output_labels, zstd_zones))),
)

input_bounds={input_labels[i]: (xmin[i], xmax[i]) for i in range(len(input_labels))}

keras_zones_surrogate = KerasSurrogate(
    keras_model=keras_zones,
    input_labels=input_labels,
    output_labels=output_labels,
    input_bounds=input_bounds,
    input_scaler=inputs_scaler,
    output_scaler=outputs_scaler,
)

## Building the Rankine Cycle Model

We now build the rankine cycle model available from DISPATCHES. This cell creates a pyomo model and uses rankine cycle functions to construct a design flowsheet. More specifically, we construct a high level flowsheet to represent the plant design. We will later construct additional flowsheets that represent plant operation for different power outputs.

In [4]:
# rankine cycle parameters
heat_recovery = True
calc_boiler_eff = True
capital_payment_years = 5
plant_lifetime = 20
coal_price = 30 #$/ton

m = ConcreteModel()

# Create capex plant
m.cap_fs = src.create_model(
    heat_recovery=heat_recovery,
    capital_fs=True, 
    calc_boiler_eff=False,
)
src.set_inputs(m.cap_fs)
src.initialize_model(m.cap_fs)
src.close_flowsheet_loop(m.cap_fs)
src.add_capital_cost(m.cap_fs)

# capital cost (M$/yr)
cap_expr = m.cap_fs.fs.capital_cost / capital_payment_years

2023-05-19 16:43:10 [INFO] idaes.init.cap_fs.fs.boiler.control_volume: Initialization Complete
2023-05-19 16:43:11 [INFO] idaes.init.cap_fs.fs.boiler: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 16:43:11 [INFO] idaes.init.cap_fs.fs.turbine: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 16:43:11 [INFO] idaes.init.cap_fs.fs.pre_condenser.control_volume: Initialization Complete
2023-05-19 16:43:11 [INFO] idaes.init.cap_fs.fs.pre_condenser: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 16:43:11 [INFO] idaes.init.cap_fs.fs.condenser.control_volume: Initialization Complete
2023-05-19 16:43:11 [INFO] idaes.init.cap_fs.fs.condenser: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 16:43:11 [INFO] idaes.init.cap_fs.fs.bfw_pump.control_volume: Initialization Complete
2023-05-19 16:43:11 [INFO] idaes.init.cap_fs.fs.bfw_pump: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 16:43:11 [INFO

## Build the grid market surrogates

We now create variables to represent the previously defined market inputs such as name-plate capacity, ramp rate, marginal cost, etc...

We use the IDAES `SurrogateBlock` to build the neural network constraints by using the previously creates `KerasSurrogate` objects. Notably, the build function automatically performs the input and output scaling because we provided the scaling information to each `KerasSurrogate`.

In [5]:
########################################
# Surrogate market inputs
########################################
m.pmax = Var(within=NonNegativeReals, bounds=(175, 450), initialize=400) #MW
m.pmin_multi = Var(within=NonNegativeReals, bounds=(0.15, 0.45), initialize=0.3)
m.ramp_multi = Var(within=NonNegativeReals, bounds=(0.5, 1.0), initialize=0.75)
m.min_up_time = Var(within=NonNegativeReals, bounds=(1.0, 16.0), initialize=4.0)
m.min_dn_multi = Var(within=NonNegativeReals, bounds=(0.5, 2.0), initialize=1.0)
m.marg_cst =  Var(within=NonNegativeReals, bounds=(5, 30), initialize=15)
m.no_load_cst =  Var(within=NonNegativeReals, bounds=(0, 2.5), initialize=1)
m.startup_cst = Var(within=NonNegativeReals, bounds=(0, 136), initialize=75)

m.pmax_con = Constraint(expr=m.pmax == m.cap_fs.fs.net_cycle_power_output * 1e-6)

# Actual generator values for minimum operating output, minimum down time, and true ramp rate
m.pmin = Expression(expr=m.pmin_multi * m.pmax) # MW
m.min_dn_time = Expression(expr=m.min_dn_multi * m.min_up_time) # hours
m.ramp_rate = Expression(expr=m.ramp_multi * (m.pmax - m.pmin)) # MW/hr

In [6]:
######################################
# Revenue surrogate
######################################
m.rev_surrogate = Var()
m.keras_revenue_surrogate = SurrogateBlock()
m.keras_revenue_surrogate.build_model(
    keras_revenue_surrogate,
    input_vars=[
        m.pmax, m.pmin_multi, m.ramp_multi, m.min_up_time,
        m.min_dn_multi, m.marg_cst, m.no_load_cst, m.startup_cst,
    ],
    output_vars = [m.rev_surrogate],
    formulation=KerasSurrogate.Formulation.REDUCED_SPACE,
)

# this is a smooth-max; it sets negative revenue to zero
m.revenue = Expression(expr=0.5*pyo.sqrt(m.rev_surrogate**2 + 0.001**2) + 0.5*m.rev_surrogate)

In [7]:
#######################################
#nstartups surrogate
#######################################
m.nstartups_surrogate = Var()
m.keras_nstartups_surrogate = SurrogateBlock()
m.keras_nstartups_surrogate.build_model(
    keras_nstartups_surrogate,
    input_vars=[
        m.pmax, m.pmin_multi, m.ramp_multi, m.min_up_time,
        m.min_dn_multi, m.marg_cst, m.no_load_cst, m.startup_cst,
    ],
    output_vars = [m.rev_surrogate],
    formulation=KerasSurrogate.Formulation.REDUCED_SPACE,
)

m.nstartups = Expression(
    expr=0.5*pyo.sqrt(m.nstartups_surrogate**2 + 0.001**2) + 0.5*m.nstartups_surrogate
)

In [8]:
############################################
# zone surrogates
############################################
m.zone_hours_surrogate = Var(range(11))
m.keras_zones_surrogate = SurrogateBlock()
m.keras_zones_surrogate.build_model(
    keras_zones_surrogate,
    input_vars=[
        m.pmax,m.pmin_multi,m.ramp_multi,m.min_up_time,
        m.min_dn_multi,m.marg_cst,m.no_load_cst,m.startup_cst
    ],
    output_vars = m.zone_hours_surrogate,
    formulation=KerasSurrogate.Formulation.REDUCED_SPACE,
)

## Build the Operation Flowsheets

We now create a flowsheet for each possible operating zone. Each flowsheet uses the market decisions defined above and we use the zone surrogates to predict the number of hours spent at each flowsheet's powr output.

In [9]:
off_fs = Block()
off_fs.fs = Block()
off_fs.fs.operating_cost = m.no_load_cst * m.pmax
off_fs.zone_hours = Expression(
    expr=0.5*pyo.sqrt(m.zone_hours_surrogate[0]**2 + 0.001**2) + 0.5*m.zone_hours_surrogate[0]
)
setattr(m, 'zone_{}'.format('off'), off_fs)

# Denote the scaled power output for each of the 10 zones (0 corresponds to pmin, 1.0 corresponds to pmax)
zone_outputs = [0.0, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 1.0]

# Create a surrogate flowsheet for each operating zone
op_zones = []
init_flag = 0
for (i, zone_output) in enumerate(zone_outputs):
    print("Creating instance ", i)
    op_fs = src.create_model(
        heat_recovery=heat_recovery,
        capital_fs=False,
        calc_boiler_eff=calc_boiler_eff,
    )
    # Set model inputs for the capex and opex plant
    src.set_inputs(op_fs)

    # Fix the p_max of op_fs to p of cap_fs for initialization
    op_fs.fs.net_power_max.fix(value(m.cap_fs.fs.net_cycle_power_output))

    #initialize with json. this speeds up model instantiation. it writes a json file \
    #for the first flowsheet which is used to initialize the next flowsheets
    if init_flag == 0:
        # Initialize the opex plant
        src.initialize_model(op_fs)

        # save model state after initializing the first instance
        init_model = to_json(op_fs.fs, return_dict=True)
        init_flag = 1
    else:
        # Initialize the capex and opex plant
        from_json(op_fs.fs, sd=init_model)

    # Closing the loop in the flowsheet
    src.close_flowsheet_loop(op_fs)
    src.add_operating_cost(op_fs, coal_price=coal_price)

    # Unfix op_fs p_max and set constraint linking that to cap_fs p_max
    op_fs.fs.net_power_max.unfix()
    op_fs.fs.eq_p_max = Constraint(
        expr=op_fs.fs.net_power_max == m.cap_fs.fs.net_cycle_power_output * 1e-6
    )

    # Fix zone power output
    op_fs.fs.eq_fix_power = Constraint(
        expr=op_fs.fs.net_cycle_power_output * 1e-6 == zone_output * (m.pmax-m.pmin) + m.pmin
    )

    # smooth max on zone hours (avoids negative hours)
    op_fs.zone_hours = Expression(
        expr=0.5*pyo.sqrt(m.zone_hours_surrogate[i+1]**2 + 0.001**2) + 0.5*m.zone_hours_surrogate[i+1]
    )

    # unfix the boiler flow rate
    op_fs.fs.boiler.inlet.flow_mol[0].setlb(0.01)
    op_fs.fs.boiler.inlet.flow_mol[0].unfix()
    setattr(m, 'zone_{}'.format(i), op_fs)
    op_zones.append(op_fs)

# scale zone hours such that they add up to 8736 (if the surrogate is good, the unscaled will be pretty close to this)
m.zone_total_hours = sum(op_zones[i].zone_hours for i in range(len(op_zones))) + off_fs.zone_hours

for op_fs in op_zones:
    op_fs.scaled_zone_hours = Var(within=NonNegativeReals, bounds=(0, 8736), initialize=100)
    # NOTE: scaled_hours_i = surrogate_i * 8736 / surrogate_total
    op_fs.con_scale_zone_hours = Constraint(
        expr=op_fs.scaled_zone_hours * m.zone_total_hours == op_fs.zone_hours * 8736
    )

off_fs.scaled_zone_hours = Var(within=NonNegativeReals, bounds=(0, 8736), initialize=100)
off_fs.con_scale_zone_hours = Constraint(
    expr=off_fs.scaled_zone_hours * m.zone_total_hours == off_fs.zone_hours * 8736
)

#operating cost in $MM (million dollars)
m.op_expr = sum(
    op_zones[i].scaled_zone_hours * op_zones[i].fs.operating_cost 
    for i in range(len(op_zones))
) * 1e-6 + off_fs.scaled_zone_hours * off_fs.fs.operating_cost * 1e-6

#startup cost in MM$
m.startup_expr = m.startup_cst * m.nstartups * m.pmax * 1e-6 #MM$

# set zone flowsheets to pyomo model
m.op_zones = op_zones

# Piecewise cost limits, connect marginal cost to operating cost. We say marginal cost is the average operating cost
m.connect_mrg_cost = Constraint(
    expr=m.marg_cst == 0.5*(op_zones[0].fs.operating_cost/m.pmin + op_zones[-1].fs.operating_cost/m.pmax)
)

# Expression for total cap and op cost - $
m.total_cost = Expression(
    expr=plant_lifetime * (m.op_expr + m.startup_expr) + capital_payment_years*cap_expr
)

# Expression for total revenue
m.total_revenue = Expression(expr=plant_lifetime*m.revenue)

# Objective $
m.obj = Objective(expr=-(m.total_revenue - m.total_cost))

# Unfixing the boiler inlet flowrate for capex plant
m.cap_fs.fs.boiler.inlet.flow_mol[0].unfix()

# Setting bounds for the capex plant flowrate
m.cap_fs.fs.boiler.inlet.flow_mol[0].setlb(0.01)

# Setting bounds for net cycle power output for the capex plant
p_lower_bound=10
p_upper_bound=500
m.cap_fs.fs.eq_min_power = Constraint(
    expr=m.cap_fs.fs.net_cycle_power_output >= p_lower_bound*1e6)

m.cap_fs.fs.eq_max_power = Constraint(
    expr=m.cap_fs.fs.net_cycle_power_output <= p_upper_bound*1e6)

Creating instance  0
2023-05-19 16:43:19 [INFO] idaes.init.fs.boiler.control_volume: Initialization Complete
2023-05-19 16:43:19 [INFO] idaes.init.fs.boiler: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 16:43:19 [INFO] idaes.init.fs.turbine: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 16:43:19 [INFO] idaes.init.fs.pre_condenser.control_volume: Initialization Complete
2023-05-19 16:43:19 [INFO] idaes.init.fs.pre_condenser: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 16:43:19 [INFO] idaes.init.fs.condenser.control_volume: Initialization Complete
2023-05-19 16:43:19 [INFO] idaes.init.fs.condenser: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 16:43:19 [INFO] idaes.init.fs.bfw_pump.control_volume: Initialization Complete
2023-05-19 16:43:19 [INFO] idaes.init.fs.bfw_pump: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 16:43:19 [INFO] idaes.init.fs.feed_water_heater.control_

## Setup Final Surrogate Inputs and Solve

We lastly fix a few surrogate inputs and solve the design problem. The solution of the design problem will determine market inputs that maximize the net plant revenue over 20 years.

In [10]:
# these are representative startup costs based on startup profiles we trained on.
# it is useful to fix the startup cost to one of these values since they are technically categorical variables
startup_csts = [0., 49.66991167, 61.09068702, 101.4374234,  135.2230393]

#fix some surrogate inputs
start_cst_index=2
m.startup_cst.fix(startup_csts[start_cst_index])
m.no_load_cst.fix(1.0)
m.min_up_time.fix(4.0)
m.min_dn_multi.fix(1.0)

solver = get_solver()
solver.options = {
    "tol": 1e-6
    #"mu_strategy": "adaptive"
}
status = solver.solve(m, tee=True)
sol_time = status['Solver'][0]['Time']

Ipopt 3.13.2: tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. S

 116r 2.0903090e+02 2.67e+03 3.24e-05  -1.5 2.25e+00    -  1.00e+00 1.00e+00h  1
 117r 2.0903090e+02 2.67e+03 1.72e+02  -3.4 3.70e+02    -  1.00e+00 1.59e-06h 20
 118r 2.1611707e+02 1.38e+05 1.68e+02  -3.4 6.50e+08    -  4.93e-04 1.98e-02f  1
 119r 2.1611727e+02 1.36e+05 3.57e+02  -3.4 3.09e+00   3.8 5.25e-03 1.17e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 120r 2.1612015e+02 1.36e+05 3.57e+02  -3.4 7.02e+08    -  1.82e-04 1.33e-05f  1
 121r 2.1611994e+02 1.36e+05 3.57e+02  -3.4 7.03e+08    -  8.80e-05 5.07e-07f  1
 122r 2.0883474e+02 3.11e+05 1.25e+03  -3.4 7.03e+08    -  1.88e-07 1.48e-02f  1
 123r 2.0162667e+02 3.15e+05 1.25e+03  -3.4 7.41e+08    -  1.79e-03 1.91e-03f  1
 124r 2.0162667e+02 3.15e+05 9.99e+02   3.3 0.00e+00    -  0.00e+00 8.04e-08R  2
 125r 2.0171952e+02 3.14e+05 9.99e+02   3.3 1.97e+06    -  4.54e-10 4.40e-06f  1
 126r 2.0158655e+02 2.62e+05 9.98e+02   1.9 1.81e+05    -  5.10e-02 6.55e-04f  1
 127r 2.0227328e+02 1.09e+04

 209r 5.8666979e+01 2.89e+04 9.99e+02   3.1 0.00e+00    -  0.00e+00 4.68e-07R  8
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 210r 4.5915157e+01 2.62e+04 1.74e+03   3.1 2.12e+05    -  7.39e-05 2.70e-03f  1
 211r 4.5927127e+01 2.61e+04 4.63e+04   3.1 6.11e+00   4.0 2.80e-01 1.22e-01f  1
 212r 4.6045710e+01 2.60e+04 1.58e+04   3.1 1.22e+00   3.5 9.90e-01 4.01e-01f  1
 213r 4.8808171e+01 4.29e+03 1.66e+04   3.1 2.11e+05    -  9.90e-01 3.15e-02f  1
 214r 5.7159080e+01 3.92e+03 1.51e+04   2.4 2.05e+05    -  1.10e-01 8.65e-02f  1
 215r 5.7233093e+01 3.75e+03 8.17e+03   2.4 1.94e+00   3.0 5.34e-01 7.52e-01f  1
 216r 5.7234017e+01 3.74e+03 3.28e+03   1.7 1.43e-01   4.4 7.20e-01 1.00e+00f  1
 217r 5.7236656e+01 3.73e+03 1.21e+03   1.7 1.53e-01   3.9 1.00e+00 1.00e+00f  1
 218r 5.7247265e+01 3.70e+03 1.02e+03   1.7 3.85e-01   3.4 1.00e+00 9.52e-01f  1
 219r 5.7263482e+01 3.60e+03 1.05e+03   1.7 1.95e+00   2.9 3.33e-01 2.56e-01f  1
iter    objective    inf_pr 

 303r 4.6771030e+01 4.87e+03 9.99e+02   2.9 0.00e+00    -  0.00e+00 3.89e-07R  5
 304r 4.1925062e+01 3.67e+03 1.35e+03   2.9 2.79e+05    -  4.10e-04 2.28e-02f  1
 305r 4.1927166e+01 3.65e+03 1.11e+04   2.9 4.31e+00   4.0 2.71e-01 2.51e-01f  1
 306r 4.1975136e+01 3.47e+03 2.79e+04   2.9 1.83e+00   3.5 3.93e-01 7.44e-01f  1
 307r 4.1975709e+01 3.46e+03 4.03e+04   2.9 4.57e-01   4.9 4.88e-01 1.00e+00f  1
 308r 4.1985819e+01 3.42e+03 3.22e+03   2.9 1.33e-01   4.4 9.67e-01 1.00e+00f  1
 309r 5.0289660e+01 7.31e+02 1.26e+04   2.9 2.72e+05    -  9.92e-01 1.36e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 310r 5.7465361e+01 7.20e+02 3.17e+03   2.2 2.35e+05    -  9.73e-01 7.16e-01f  1
 311r 1.1737769e+02 7.07e+02 8.24e+02   1.5 4.12e+05    -  5.71e-01 7.45e-01f  1
 312r 1.2054145e+02 8.16e+02 5.09e+02   0.8 8.78e+05    -  5.44e-01 4.11e-01f  1
 313r 1.2680237e+02 1.26e+03 7.13e+02   0.8 2.26e+06    -  3.68e-01 1.43e-01f  1
 314r 1.1434041e+02 5.50e+03

 417r-1.9939207e+02 2.15e+04 9.99e+02   2.4 0.00e+00    -  0.00e+00 3.08e-07R  4
 418r-1.9862009e+02 3.05e+03 9.94e+02   2.4 8.37e+05    -  8.32e-02 5.34e-03f  1
 419r-1.9354186e+02 2.28e+03 1.02e+03   2.4 2.45e+05    -  1.76e-02 3.32e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 420r-1.7944941e+02 4.10e+02 1.24e+03   2.4 2.36e+05    -  1.31e-01 1.33e-01f  1
 421r-1.7934667e+02 3.49e+02 4.26e+03   2.4 2.03e+01   2.0 5.06e-02 7.33e-02f  1
 422r-1.7934407e+02 3.38e+02 5.29e+03   2.4 8.06e-01   4.2 2.49e-01 3.12e-01h  1
 423r-1.7934171e+02 3.26e+02 1.28e+05   2.4 1.03e+00   4.7 9.90e-01 7.49e-01f  1
 424r-1.7934120e+02 3.24e+02 2.10e+05   2.4 1.52e+00   5.1 1.27e-01 4.31e-01f  1
 425r-1.7934116e+02 3.24e+02 7.69e+05   2.4 3.42e-01   6.4 9.90e-01 7.33e-01h  1
 426r-1.7934114e+02 3.24e+02 5.43e+05   2.4 6.94e-02   6.8 8.36e-01 6.22e-01h  1
 427r-1.7934108e+02 3.23e+02 2.01e+05   2.4 4.00e-02   6.4 1.00e+00 1.00e+00f  1
 428r-1.7934090e+02 3.23e+02

...

 630r-2.1676418e+01 4.08e+01 1.03e+02  -6.5 1.84e+01  -3.4 2.40e-01 6.94e-04f  1
 631r-2.1728085e+01 4.08e+01 1.30e+02  -6.5 1.21e+03  -3.9 4.99e-02 1.25e-04h  1
 632r-1.6251835e+01 4.08e+01 3.36e+01  -6.5 2.45e+01  -3.5 1.00e+00 6.41e-01f  1
 633r-1.2189075e+01 4.08e+01 1.93e+02  -6.5 1.63e+01  -3.0 1.00e+00 7.07e-01f  1
 634r-1.1964986e+01 4.08e+01 1.28e+02  -6.5 3.28e+00  -1.7 1.14e-01 1.90e-01f  1
 635r-1.1964986e+01 4.08e+01 1.28e+02  -6.5 4.08e-01   3.2 5.20e-06 2.89e-04f  1
 636r-1.1964986e+01 4.08e+01 2.96e+02  -6.5 2.97e-02   3.7 1.00e+00 1.94e-06f  2
 637r-1.1964986e+01 4.08e+01 2.06e+02  -6.5 1.13e-03   4.1 1.00e+00 1.40e-02f  1
 638r-1.1964983e+01 4.08e+01 1.76e+02  -6.5 4.67e-03   3.6 1.00e+00 5.37e-01f  1
 639r-1.0438751e+01 7.68e+02 1.89e+02  -6.5 9.17e+07    -  3.86e-01 9.59e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 640r-1.0438409e+01 7.52e+02 9.00e+02  -6.5 6.73e-02   4.0 1.72e-01 1.99e-02h  1
 641r-1.0433752e+01 5.44e+02

 769r-1.1165334e+01 2.76e+02 3.32e+02  -6.5 7.22e+01  -1.5 3.10e-06 1.46e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 770r-1.1165334e+01 2.76e+02 1.32e+03  -6.5 2.58e+01  -1.1 7.02e-01 1.06e-07h  1
 771r-1.1165334e+01 2.76e+02 6.81e+02  -6.5 7.87e+01  -1.6 3.45e-01 1.16e-07h  1
 772r-1.1160900e+01 2.75e+02 6.31e+02  -6.5 2.39e+02  -2.1 5.34e-08 4.06e-03f  1
 773r-1.1160897e+01 2.75e+02 6.31e+02  -6.5 1.66e+00   0.2 6.39e-01 7.29e-04h  1
 774r-1.1148228e+01 4.06e+01 1.25e+01  -6.5 3.77e+00  -0.3 1.00e+00 1.00e+00f  1
 775r-1.1128012e+01 4.06e+01 4.34e+01  -6.5 1.04e+01  -0.8 4.53e-01 5.09e-01f  1
 776r-1.1118199e+01 4.06e+01 1.70e+02  -6.5 4.73e+00  -0.3 1.29e-03 6.41e-01f  1
 777r-1.1116660e+01 1.28e+02 5.63e+02  -6.5 1.63e+02  -0.8 7.98e-03 4.22e-02f  1
 778r-1.1116660e+01 1.28e+02 5.63e+02  -6.5 7.52e+00  -0.4 0.00e+00 2.50e-11R 12
 779r-1.1115325e+01 1.25e+02 5.51e+02  -6.5 1.39e+01  -0.9 1.00e+00 2.13e-02f  1
iter    objective    inf_pr 

 872 -1.3725059e+00 3.85e+03 1.32e+06  -1.0 2.31e+08    -  1.84e-07 4.21e-03f  6
 873r-1.3725059e+00 3.85e+03 9.99e+02   2.0 0.00e+00    -  0.00e+00 3.91e-07R  7
 874r-8.7215869e-01 8.69e+02 9.99e+02   2.0 2.45e+05    -  1.24e-05 9.91e-04f  1
 875 -6.4260566e-01 3.65e+03 1.15e+02  -1.0 1.43e+08    -  4.42e-03 4.42e-03s 15
 876r-6.4260566e-01 3.65e+03 9.99e+02   1.9 0.00e+00    -  0.00e+00 0.00e+00R  1
 877r 3.0718000e-02 9.22e+02 9.99e+02   1.9 3.19e+05    -  6.69e-05 9.91e-04f  1
 878r 6.6419154e-02 5.66e+02 1.08e+03   1.9 2.55e+04    -  1.40e-01 1.17e-03f  1
 879r 6.5122646e+00 2.08e+02 8.54e+02   1.9 5.29e+03    -  9.44e-01 1.41e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 880  6.5292457e+00 2.10e+02 1.53e+03  -1.0 1.50e+08    -  5.73e-02 3.60e-04h  1
 881  6.5712919e+00 2.12e+02 1.81e+04  -1.0 2.30e+08    -  5.46e-02 3.12e-04h  5
 882  6.5878718e+00 2.13e+02 3.35e+04  -1.0 2.30e+08    -  5.73e-02 1.23e-04h 10
 883  6.5960879e+00 2.13e+02

 965  2.6899993e+02 6.70e+06 4.47e+04  -1.0 2.95e+08    -  1.54e-01 1.75e-03w  1
 966  5.5582339e+01 2.70e+01 4.13e+04  -1.0 1.15e+08    -  9.67e-03 7.35e-05h 13
 967  5.5616784e+01 2.70e+01 4.24e+04  -1.0 3.02e+08    -  4.62e-02 7.45e-05h 14
 968  5.5650801e+01 2.70e+01 6.96e+04  -1.0 3.03e+08    -  1.00e+00 7.37e-05h 14
 969  5.5685845e+01 2.70e+01 6.99e+04  -1.0 2.98e+08    -  9.37e-03 7.49e-05h 14
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 970  5.5720873e+01 2.70e+01 7.19e+04  -1.0 3.01e+08    -  4.68e-02 7.54e-05h 14
 971  5.5755831e+01 2.70e+01 7.24e+04  -1.0 3.01e+08    -  1.27e-02 7.54e-05h 14
 972  5.5790676e+01 2.70e+01 1.16e+05  -1.0 3.01e+08    -  1.00e+00 7.52e-05h 14
 973  5.5825767e+01 2.70e+01 1.17e+05  -1.0 2.96e+08    -  9.68e-03 7.48e-05h 14
 974  5.5860830e+01 2.70e+01 1.20e+05  -1.0 3.00e+08    -  4.86e-02 7.55e-05h 14
 975  5.5895826e+01 2.70e+01 1.93e+05  -1.0 3.00e+08    -  1.00e+00 7.54e-05h 14
 976  3.4840961e+02 8.43e+06

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1060  3.2820788e+02 8.12e+01 7.28e+00  -2.5 1.73e+05 -10.8 1.00e+00 1.00e+00f  1
1061  3.0911770e+02 3.65e+03 1.36e+02  -2.5 1.83e+07    -  1.00e+00 1.00e+00f  1
1062  3.2815512e+02 3.55e+02 1.46e+01  -2.5 3.46e+05 -11.3 1.00e+00 1.00e+00h  1
1063  3.1276588e+02 3.11e+03 5.54e+01  -2.5 5.80e+06    -  2.89e-01 4.45e-01f  1
1064  3.3365584e+02 6.37e+02 1.97e+01  -2.5 1.71e+05 -10.9 1.00e+00 1.00e+00h  1
1065  3.1447219e+02 1.64e+03 1.61e+02  -2.5 8.86e+05 -11.4 5.39e-01 1.00e+00f  1
1066  3.2139869e+02 4.38e+02 1.99e+02  -2.5 1.08e+06 -11.8 1.00e+00 1.00e+00h  1
1067  3.2507697e+02 2.56e+02 4.18e+01  -2.5 6.04e+05 -11.4 1.00e+00 1.00e+00h  1
1068  3.2320339e+02 9.40e+01 1.51e+01  -2.5 1.74e+05 -11.9 1.00e+00 1.00e+00h  1
1069  3.1263875e+02 1.49e+00 8.74e+02  -2.5 3.68e+07    -  1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1070  3.0458683e+02 1.52e+00

## Display Solution

In [11]:
#get value of surrogate inputs
x = [value(m.pmax),
    value(m.pmin_multi),
    value(m.ramp_multi),
    value(m.min_up_time),
    value(m.min_dn_multi),
    value(m.marg_cst),
    value(m.no_load_cst),
    value(m.startup_cst)
    ]

print("Market Inputs:", x)

#calculate revenues, costs, etc...
optimal_objective = -value(m.obj)

zone_hours = [value(m.zone_off.zone_hours)]
scaled_zone_hours = [value(m.zone_off.scaled_zone_hours)]
op_cost = [value(m.zone_off.fs.operating_cost)]
op_expr = value(m.zone_off.fs.operating_cost) # in dollars [$]
for zone in m.op_zones:
    zone_hours.append(value(zone.zone_hours))
    scaled_zone_hours.append(value(zone.scaled_zone_hours))
    op_cost.append(value(zone.fs.operating_cost))
    op_expr += value(zone.scaled_zone_hours)*value(zone.fs.operating_cost)

#more calculations of revenue and cost
revenue_per_year = value(m.revenue)
cap_expr = value(m.cap_fs.fs.capital_cost)/capital_payment_years
startup_expr = value(m.startup_expr)
total_cost = plant_lifetime*op_expr/1e6 + capital_payment_years*cap_expr
total_revenue = plant_lifetime*revenue_per_year

print("Annual Revenue [MM$]: ", revenue_per_year)
print("Capital Cost [MM$]: ",cap_expr)
print("Annual Startup Cost [MM$]: ", startup_expr)
print("Total Cost (20 years): ", total_cost)
print("Net Revenue (20 years)", optimal_objective)

Market Inputs: [259.91910248189976, 0.4423848332093146, 0.6392249722616824, 4.0, 1.0, 17.03405649932845, 1.0, 61.09068702]
Annual Revenue [MM$]:  42.350552833052404
Capital Cost [MM$]:  109.78197558234294
Annual Startup Cost [MM$]:  2.4502773280683162e-11
Total Cost (20 years):  1166.2990079394763
Net Revenue (20 years) -321.72662923377106
